<a href="https://colab.research.google.com/github/EvanM19/Projet-scoring/blob/main/descriptive%20statistics/Feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature_selection and descriptive analisis



### Data and library importation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from PyPDF2 import PdfMerger

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

In [ ]:


samples = pd.read_excel('../data/samples.xlsx', sheet_name=None)

colonnes_a_supprimer = [ 'id_client', 'CODTYPPRS', 'CRTOD_CMT_PRO_IND_0209','INDPARDCD','CRTOP_AG_IND_0038', 'CRTOP_IND_0038', 'CRTOU_CCT_PRO_IND_0209', 'DATCREENT', 'DATRLNORG', 'RATIO1', 'RATIO3', 'TOP_PRO_ACTIF', 'IND_ACTIF', 'IND_PER_PM', 'CODCMNIEE', 'CODPAY_NAISSANCE', 'CODPAY_NATIONALITE', 'CODPAY_RESIDENCE', 'CODSEXPRS', 'secteur_activite', 'secteur_activite_bis' ]

samples = samples
train = samples[list(samples.keys())[0]]
train = train.drop(columns=colonnes_a_supprimer)
test = samples[list(samples.keys())[1]]
test = test.drop(columns=colonnes_a_supprimer)
oot = samples[list(samples.keys())[2]]
oot = oot.drop(columns=colonnes_a_supprimer)
sample = pd.concat([train, test, oot], ignore_index=True)

### Preprocesing

In [67]:
# print(sample.shape)
# print(sample.dtypes)
# sample.describe()
# print(sample.isnull().sum())

# print(sample['DATCREPRSENT'].head())
# print(sample['DATCREPRSENT'].describe())
# print(sample['DATCREPRSENT'].isnull().mean())
# print("25,76 % de valeurs manquantes. Par la suite on regroupera par période et on créera une autre modalité pour les valeurs manquantes.")

In [3]:
#####Datehis

# print(sample['datdelhis'].unique())
sample['datdelhis'] = sample['datdelhis'] % 100
# print(sample['datdelhis'].unique())
sample['datdelhis'] = sample['datdelhis'].astype(str)

# observations_par_tranche = sample.groupby('datdelhis').size()
# observations_par_tranche.plot(kind='bar', color='skyblue')
# plt.title("Nombre d'observations par tranche de temps")
# plt.xlabel("Temps")
# plt.ylabel("Nombre d'observations")
# plt.xticks(rotation=45)
# plt.show()

quali=['DDefaut_NDB','datdelhis','AGEPRS','CODITDBDF_PRO','CODETAJUR','CODNAF2']


### Plots

In [ ]:
o="plots"
os.makedirs(o,exist_ok=True)

def f(data, target_column,feature,graph):
  plt.figure(figsize=(10, 6))
  if data[feature].dtype in ['int64', 'float64'] or feature not in quali:
    if graph==1:
      sns.kdeplot(data=data[data[target_column] == 0], x=feature, label='Cible = 0', fill=True, alpha=0.5)
      sns.kdeplot(data=data[data[target_column] == 1], x=feature, label='Cible = 1', fill=True, alpha=0.5)
      plt.title(f"Distribution de {feature} par {target_column}")
      plt.xlabel(feature)
      plt.ylabel("Densité")
    
      file_path = os.path.join(o, "c.pdf")
      plt.savefig(file_path)
      plt.close()
    else:
      plt.subplot(1, 2, 1)
      a = data[data[feature] != 0]
      a = a.sort_values(by=feature)
      a = a[int(len(a) * 0.01):int(len(a) * 0.99)]
      sns.kdeplot(data=data[data[target_column] == 0], x=feature, label='Cible = 0', fill=True, alpha=0.5)
      sns.kdeplot(data=data[data[target_column] == 1], x=feature, label='Cible = 1', fill=True, alpha=0.5)
      plt.title(f"Distribution de {feature} avec 0")
      plt.xlabel(feature)
      plt.ylabel("Densité")
      plt.subplot(1, 2, 2)
      sns.kdeplot(data=a[a[target_column] == 0], x=feature, label='Cible = 0', fill=True, alpha=0.5)
      sns.kdeplot(data=a[a[target_column] == 1], x=feature, label='Cible = 1', fill=True, alpha=0.5)
      plt.title(f"Distribution de {feature} sans 0")
      plt.xlabel(feature)
      plt.ylabel("Densité")
    
      file_path = os.path.join(o, "c.pdf")
      plt.savefig(file_path)
      plt.close()
  else:
    counts_0 = data[data[target_column] == 0][feature].value_counts(normalize=True)
    counts_1 = data[data[target_column] == 1][feature].value_counts(normalize=True)
    categories = list(set(counts_0.index).union(set(counts_1.index)))
    counts_0 = counts_0.reindex(categories, fill_value=0)
    counts_1 = counts_1.reindex(categories, fill_value=0)
    x = range(len(categories))
    plt.bar(x, counts_0, width=0.4, label='Cible = 0', align='center', alpha=0.7)
    plt.bar([xi + 0.4 for xi in x], counts_1, width=0.4, label='Cible = 1', align='center', alpha=0.7)
    plt.xticks([xi + 0.2 for xi in x], categories, rotation=45)
    plt.title(f"Distribution de {feature} par {target_column}")
    plt.xlabel(feature)
    plt.ylabel("Proportion")
    plt.legend()
    plt.tight_layout()
    # plt.show()
    file_path = os.path.join(o, "c.pdf")
    plt.savefig(file_path)
    plt.close()


def g(data, target_column):
    features = [col for col in data.columns if col != target_column]
    features=features[1:]
    for x in features:

        freq = sample[x].value_counts(normalize=True)
        top_modality = freq.idxmax()
        graph=1
        if top_modality==0 and freq.max() >0.75:
          graph=2
          plt.figure(figsize=(12, 6))
          plt.subplot(1, 2, 1)
          a=sample[x]
          a = a.sort_values()
          a = a[int(len(a) * 0.01):int(len(a) * 0.99)]
          sns.histplot(a, bins=30, color='blue')
          plt.title('avec 0')
          plt.subplot(1, 2, 2)
          a = sample[sample[x] != 0]
          a = a.sort_values(by=x)
          a = a[int(len(a) * 0.01):int(len(a) * 0.99)]
          sns.histplot(a[x], bins=30, color='blue')
          plt.title('Sans 0')
          file_path = os.path.join(o, "a.pdf")
          plt.savefig(file_path)
          plt.close()
        

          a = sample.dropna(subset=[x, 'datdelhis'])
          df = a[a[x]!=0]
          df = df.sort_values('datdelhis')
          plt.figure(figsize=(10, 6))
          plt.subplot(1, 2, 1)
          sns.boxplot(x='datdelhis', y=x,showfliers=False, data=df, palette='pastel')
          plt.xlabel('Date (Datdelhis)')
          plt.ylabel(x)
          plt.title(f'Distribution de {x} par Date (Sans 0)')
          df = sample[sample[x] == 0]
          df = df.sort_values('datdelhis')
          observations_par_date = df.groupby('datdelhis').size().reset_index(name='Nombre d\'observations')
          plt.subplot(1, 2, 2)
          sns.barplot(x='datdelhis', y='Nombre d\'observations', data=observations_par_date, palette='pastel')
          plt.title(f'Nb d\'observations avec {x} nul par Date')
          plt.xlabel('Date')
          plt.ylabel(f'Nombre d\'observations {x}')
        
          file_path = os.path.join(o, "b.pdf")
          plt.savefig(file_path)
          plt.close()
        else:
          a = sample[x]
          a = a.sort_values()
          a = a[int(len(a) * 0.01):int(len(a) * 0.99)]
          plt.figure(figsize=(12, 6))
          sns.histplot(a, bins=30, color='blue')
        
          file_path = os.path.join(o, "a.pdf")
          plt.savefig(file_path)
          plt.close()

          a = sample.dropna(subset=[x, 'datdelhis'])
          df = a.sort_values('datdelhis')
          plt.figure(figsize=(10, 6))
          sns.boxplot(x='datdelhis',y=x, data=df,showfliers=False, palette='pastel')
          plt.xlabel('Datdelhis')
          plt.ylabel(x)
        
          file_path = os.path.join(o, "b.pdf")
          plt.savefig(file_path)
          plt.close()

        if x=='CODITDBDF_PRO':
          graph=1
        f(train,target_column,x,graph)
        graph=1

        merger = PdfMerger()
        pdf_list=["plots/a.pdf","plots/b.pdf","plots/c.pdf"]
        for pdf in pdf_list:
          merger.append(pdf)
        merger.write(f"plots/{x}.pdf")
        merger.close()

g(sample,'DDefaut_NDB')